In [ ]:
# default_exp reward

# Reward

> Rewards - non-differentiable scores for samples

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export

from mrl.imports import *
from mrl.core import *
from mrl.callbacks import *
from mrl.torch_imports import *
from mrl.torch_core import *

In [ ]:
# export

class Reward(Callback):
    def __init__(self, name, sample_name='samples', 
                 weight=1., bs=None,
                 order=10, track=True, log=True):
        super().__init__(name=name, order=order)
        
        self.sample_name = sample_name
        self.weight = weight
        self.bs = bs
        self.track = track
        self.log = log
        self.score_log = {}
        
    def load_data(self, samples, values):
        for i in range(len(samples)):
            self.score_log[samples[i]] = values[i]
        
    def setup(self):
        log = self.environment.log
        log.add_log(self.name)
        if self.track:
            log.add_metric(self.name)
            
    def _compute_reward(self, samples):
        return [0. for i in samples]
    
    def compute_batched_reward(self, samples):
        if self.bs is not None:
            sample_chunks = chunk_list(samples, self.bs)
            rewards = []
            for chunk in sample_chunks:
                rewards_iter = self._compute_reward(chunk)
                if isinstance(rewards_iter, torch.Tensor):
                    rewards_iter = rewards_iter.detach().cpu()
                    
                rewards += list(rewards_iter)
            
        else:
            rewards = self._compute_reward(samples)
            if isinstance(rewards, torch.Tensor):
                rewards = rewards.detach().cpu()
            
        return rewards
    
    def compute_reward(self):
        env = self.environment
        batch_state = env.batch_state
        samples = batch_state[self.sample_name]
#         prescored = batch_state.prescored
        
        rewards = np.array([0. for i in samples])
        
        to_score = []
        to_score_idxs = []
        
        for i in range(len(samples)):
#             if not prescored[i]:
                
            if self.log:
                if samples[i] in self.score_log:
                    rewards[i] = self.score_log[samples[i]]
                else:
                    to_score.append(samples[i])
                    to_score_idxs.append(i)

            else:
                to_score.append(samples[i])
                to_score_idxs.append(i)
                    
        new_rewards = self.compute_batched_reward(to_score)
        
        for i in range(len(to_score)):
            batch_idx = to_score_idxs[i]
            reward = new_rewards[i]
            rewards[batch_idx] = reward
            
            if self.log:
                self.score_log[to_score[i]] = reward
                
        rewards = to_device(torch.tensor(rewards).float()).squeeze()
        rewards = rewards * self.weight
        
        batch_state.rewards += rewards
        batch_state[self.name] = rewards
        
        if self.track:
            env.log.update_metric(self.name, rewards.mean().detach().cpu().numpy())
        

In [ ]:
# export

class RewardModification(Callback):
    def __init__(self, name, sample_name='samples', 
                 weight=1., bs=None,
                 order=10, track=True):
        super().__init__(name=name, order=order)
        
        self.sample_name = sample_name
        self.weight = weight
        self.bs = bs
        self.track = track
        
    def setup(self):
        log = self.environment.log
        log.add_log(self.name)
        if self.track:
            log.add_metric(self.name)
            
    def _compute_reward_modification(self, samples):
        return [0. for i in samples]
    
    def compute_batched_reward_modification(self, samples):
        if self.bs is not None:
            sample_chunks = chunk_list(samples, self.bs)
            rewards = []
            for chunk in sample_chunks:
                rewards_iter = self._compute_reward_modification(chunk)
                if isinstance(rewards_iter, torch.Tensor):
                    rewards_iter = rewards_iter.detach().cpu()
                    
                rewards += list(rewards_iter)
            
        else:
            rewards = self._compute_reward_modification(samples)
            if isinstance(rewards, torch.Tensor):
                rewards = rewards.detach().cpu()
            
        return rewards
    
    def reward_modification(self):
        env = self.environment
        batch_state = env.batch_state
        samples = batch_state[self.sample_name]
        
        rewards = self.compute_batched_reward_modification(samples)
                
        rewards = to_device(torch.tensor(rewards).float()).squeeze()
        rewards = rewards * self.weight
        
        batch_state.rewards_final += rewards
        batch_state[self.name] = rewards
        
        if self.track:
            env.log.update_metric(self.name, rewards.mean().detach().cpu().numpy())

In [ ]:
# export

class ContrastiveReward(Reward):
    def __init__(self, base_reward, max_score=None):
        super().__init__(name = base_reward.name,
                         sample_name = base_reward.sample_name,
                         weight = base_reward.weight,
                         bs = base_reward.bs,
                         order = base_reward.order,
                         track = base_reward.track,
                         log = base_reward.log)
        
        self.base_reward = base_reward
        self.max_score = max_score
        
        
    def __call__(self, event_name):
        
        event = getattr(self, event_name, None)
        
        if event is not None:
            output = event()
        else:
            output = None
            
        if not event_name=='compute_reward':
            _ = self.base_reward(event_name)
            
        return output
        
    def compute_and_clean(self, samples):
        rewards = self.base_reward._compute_reward(samples)
        if isinstance(rewards, torch.Tensor):
            rewards = rewards.detach().cpu()
            
        rewards = np.array(rewards)
        return rewards
        
    def _compute_reward(self, samples):
        source_samples = [i[0] for i in samples]
        target_samples = [i[1] for i in samples]
        
        source_rewards = self.compute_and_clean(source_samples)
        target_rewards = self.compute_and_clean(target_samples)
        
        rewards = target_rewards - source_rewards
        if self.max_score is not None:
            rewards = rewards / (self.max_score-source_rewards)
            
        return list(rewards)

In [ ]:
# export

class FunctionReward(Reward):
    def __init__(self, reward_function, name, sample_name='samples', 
                 weight=1., bs=None,
                 order=10, track=True, log=True):
        super().__init__(name=name, 
                         sample_name=sample_name, 
                         weight=weight,
                         bs=bs,
                         order=order, 
                         track=track, 
                         log=log)
        
        self.reward_function = reward_function
        
        
    def _compute_reward(self, samples):
        rewards = []
        if samples:
            rewards = self.reward_function(samples)
        return rewards
        
        


In [ ]:
# export

class NoveltyReward(RewardModification):
    def __init__(self, weight=1., track=True):
        super().__init__(name='novel', 
                         sample_name='samples',
                         weight=weight, 
                         order=10, 
                         track=track)
        
        
    def _compute_reward_modification(self, samples):
        
        df = self.environment.log.df
        
        new = (~pd.Series(samples).isin(df.samples)).values

        new = [float(i) for i in new]
        return new